<div align="center">

# <span style="color: #3498db;">CA2 - Game</span>

**<span style="color:rgb(247, 169, 0);">Parsa Saeednia</span> - <span style="color:rgb(143, 95, 195);">810102460</span>**

</div>


# <span style="color: #3498db;">Minmax Algorithm</span>

In [1]:
import random
import itertools
import numpy as np
from math import inf
import time
import statistics
import time
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.13.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
eval_config = {"Win" : 10000, 
               "Lost" : -10000, 
               "Draw" : 0}

center_value=5
corner_value=3

In [3]:
class PentagoGame:
    def __init__(self, ui=False, print=False, depth=2, use_alpha_beta_=False):
        self.board = np.zeros((6, 6), dtype=int)
        self.current_player = 1
        self.ui = ui
        self.depth = depth
        self.nodes_visited = 0
        self.game_over = False
        self.result = None
        self.selected_block = None
        self.move_stage = 0  # 0: place piece, 1: select block, 2: rotate
        self.temp_piece = None
        self.print = print
        self.use_alpha_beta = use_alpha_beta_
        self.total_visited_nodes = 0

        if ui:
            pygame.font.init()
            self.screen = pygame.display.set_mode((800, 600))
            pygame.display.set_caption("Pygame Board")
            # self.font = pygame.font.SysFont("Arial", 20)
            self.show_buttons = False
            self.buttons = {
                "rotate_cw": pygame.Rect(650, 200, 100, 50),
                "rotate_ccw": pygame.Rect(650, 300, 100, 50),
            }
            self.setup_controls()
            self.draw_board()

    def setup_controls(self):
        if self.show_buttons:
            pygame.draw.rect(self.screen, (144, 238, 144), self.buttons["rotate_cw"])   # Light Green
            pygame.draw.rect(self.screen, (173, 216, 230), self.buttons["rotate_ccw"])  # Light Blue

            self.screen.draw_text("CLOCKWISE", self.buttons["rotate_cw"].center)
            self.screen.draw_text("COUNTER-CLOCKWISE", self.buttons["rotate_ccw"].center)



    def hide_rotation_buttons(self):
        self.show_buttons = False

    def show_rotation_buttons(self):
        self.show_buttons = True

    def copy_board(self, board):
        return np.copy(board)

    def rotate_block(self, board, block, direction):
        row_start = (block // 2) * 3
        col_start = (block % 2) * 3
        sub = board[row_start : row_start + 3, col_start : col_start + 3]
        rotated = np.rot90(sub, 3 if direction == "cw" else 1)
        board[row_start : row_start + 3, col_start : col_start + 3] = rotated

    def get_possible_moves(self, board, player):
        moves = []
        for i in range(6):
            for j in range(6):
                if board[i][j] == 0:
                    for block in range(4):
                        for dir in ["cw", "ccw"]:
                            moves.append((i, j, block, dir))
        return moves

    def apply_move(self, board, move, player):
        new_board = self.copy_board(board)
        row, col, block, direction = move
        if new_board[row][col] != 0:
            return None
        new_board[row][col] = player
        self.rotate_block(new_board, block, direction)
        return new_board

    def check_winner(self, board):
        for i in range(6):
            for j in range(6):
                if board[i][j] == 0:
                    continue

                # Horizontal
                if j <= 1 and np.all(board[i, j : j + 5] == board[i][j]):
                    return board[i][j]

                # Vertical
                if i <= 1 and np.all(board[i : i + 5, j] == board[i][j]):
                    return board[i][j]

                # Diagonal
                if (
                    i <= 1
                    and j <= 1
                    and all(board[i + k][j + k] == board[i][j] for k in range(5))
                ):
                    return board[i][j]

                # Anti-diagonal
                if (
                    i <= 1
                    and j >= 4
                    and all(board[i + k][j - k] == board[i][j] for k in range(5))
                ):
                    return board[i][j]
        if np.all(board != 0):
            return 0
        return None

    def evaluate_function(self, current_board):        
        winner = self.check_winner(current_board)
        
        if winner == -1: return eval_config["Win"]
        if winner == 1: return eval_config["Lost"]
        if winner == 0: return eval_config["Draw"]
        
        score = 0
        
        positional_scores = [
            [2, 1, 2, 2, 1, 2], 
            [1, 3, 1, 1, 3, 1], 
            [2, 1, 2, 2, 1, 2], 
            [2, 1, 2, 2, 1, 2], 
            [1, 3, 1, 1, 3, 1], 
            [2, 1, 2, 2, 1, 2]
        ]
        
        lines = []
        
        for i in range(6):
            for j in range(2):
                lines.append(current_board[i, j:j+5])
                lines.append(current_board[j:j+5, i])
        
        
        for i in range(2):
            for j in range(2):
                lines.append(np.array([current_board[i+k, j+k] for k in range(5)]))  # Main diagonal
                lines.append(np.array([current_board[i+k, 5-j-k] for k in range(5)]))  # Anti-diagonal
        
        
        for i in range(6):
            for j in range(6):
                if current_board[i][j] == -1:
                    score += positional_scores[i][j]
                elif current_board[i][j] == 1:
                    score -= positional_scores[i][j]
        
        return score

    def evaluate_line(self, line):
        ai_count = np.count_nonzero(line == -1)
        player_count = np.count_nonzero(line == 1)
        empty_count = 5 - (ai_count + player_count)
        
        score = 0
        
        if ai_count == 4 and empty_count == 1: score += 1000
        elif ai_count == 3 and empty_count == 2: score += 50
        elif ai_count == 2 and empty_count == 3: score += 10
        elif ai_count == 1 and empty_count == 4: score += 2
        
        if player_count == 4 and empty_count == 1: score -= 1025
        elif player_count == 3 and empty_count == 2: score -= 55
        elif player_count == 2 and empty_count == 3: score -= 12
        elif player_count == 1 and empty_count == 4: score -= 3
        
        if ai_count == 2 and player_count == 0: score += 5
        if player_count == 2 and ai_count == 0: score -= 5
        
        return score
    
    
    def check_terminal(self, current_board, current_depth):
        winner = self.check_winner(current_board)
        if winner != None:
            if winner == -1:
                return eval_config["Win"] - (self.depth - current_depth)
            elif winner == 1:
                return eval_config["Lost"] - (self.depth - current_depth)
            else:
                return 0
            
        return None
    
    
    def minimax(self, current_board, current_depth, min_flag=True):
        self.nodes_visited += 1
        
        winner = self.check_terminal(current_board, current_depth)
        if winner != None:
            return winner

        if current_depth <= 0:
            return self.evaluate_function(current_board)
        
        optimal_value = inf if min_flag else -inf
        
        for move in self.get_possible_moves(current_board, 1 if min_flag else -1):
            new_board = self.apply_move(current_board, move, 1 if min_flag else -1)
            if new_board is None:
                continue
            
            value = self.minimax(new_board, current_depth - 1, not min_flag)
            optimal_value = min(optimal_value, value) if min_flag else max(optimal_value, value)
            
        return optimal_value
    
    
    def minimax_alpha_beta(self, current_board, current_depth, alpha=-inf, beta=inf, min_flag=True):
        self.nodes_visited += 1
        
        winner = self.check_terminal(current_board, current_depth)
        if winner != None:
            return winner
        
        if current_depth <= 0:
            return self.evaluate_function(current_board)
        
        optimal_value = inf if min_flag else -inf
        for move in self.get_possible_moves(current_board, 1 if min_flag else -1):
            new_board = self.apply_move(current_board, move, 1 if min_flag else -1)
            
            if new_board is None:
                continue
            
            value = self.minimax_alpha_beta(new_board, current_depth - 1, alpha, beta, not min_flag)
            optimal_value = min(optimal_value, value) if min_flag else max(optimal_value, value)
            if min_flag:
                if optimal_value <= alpha:
                    return optimal_value
                
                beta = min(optimal_value, beta)
            else:
                if optimal_value >= beta:
                    return optimal_value
                
                alpha = max(optimal_value, alpha)
            
        return optimal_value
    

    def get_computer_move(self):
        start_time = time.time()
        best_move = None
        best_value = -inf
        alpha = -inf
        beta = inf
        
        moves = self.get_possible_moves(self.board, -1)
        if not moves:
            return None
        best_move = moves[0]

        for move in moves:
            if self.game_over:
                break
            new_board = self.apply_move(self.board, move, -1)
            if new_board is None:
                continue

            try:
                if self.use_alpha_beta:
                    value = self.minimax_alpha_beta(new_board, self.depth - 1, alpha, beta)
                else:
                    value = self.minimax(new_board, self.depth - 1)
                    
            except:
                value = -inf
                
            if value > best_value:
                best_value = value
                best_move = move
                
            if self.use_alpha_beta:
                if alpha < best_value:
                    alpha = best_value
                
                
        if self.print:
            print(f"Move took {time.time()-start_time:.2f}s, nodes visited: {self.nodes_visited}")
        self.total_visited_nodes += self.nodes_visited
        self.nodes_visited = 0
        return best_move
    

    def draw_text(self, text, center_pos, max_width):
        font_size = 24
        font = pygame.font.Font(None, font_size)
        text_surface = font.render(text, True, (0, 0, 0))

        text_width = text_surface.get_width()
        if text_width > max_width:
            scale_factor = max_width / text_width
            new_font_size = int(font_size * scale_factor)
            font = pygame.font.Font(None, new_font_size)
            text_surface = font.render(text, True, (0, 0, 0))

        text_rect = text_surface.get_rect(center=center_pos)
        self.screen.blit(text_surface, text_rect)

    def draw_board(self):
        self.screen.fill((0, 0, 0))

        for i in range(6):
            for j in range(6):
                x0 = j * 100
                y0 = i * 100

                if self.board[i][j] == 1:
                    pygame.draw.circle(self.screen, (255, 0, 0), (x0 + 50, y0 + 50), 40)
                elif self.board[i][j] == -1:
                    pygame.draw.circle(self.screen, (0, 0, 255), (x0 + 50, y0 + 50), 40)

                pygame.draw.rect(self.screen, (255, 255, 255), (x0, y0, 100, 100), 1)

        for i in [3, 6]:
            pygame.draw.line(self.screen, (255, 255, 255), (0, i * 100), (600, i * 100), 3)  # Horizontal
            pygame.draw.line(self.screen, (255, 255, 255), (i * 100, 0), (i * 100, 600), 3)  # Vertical

        # Show rotation buttons if in move_stage 2
        if self.move_stage == 2:
            self.highlight_selected_block()
            self.show_rotation_buttons()

        if self.show_buttons:
            pygame.draw.rect(self.screen, (144, 238, 144), self.buttons["rotate_cw"])  # Light Green
            pygame.draw.rect(self.screen, (173, 216, 230), self.buttons["rotate_ccw"])  # Light Blue

            self.draw_text(
                "CLOCKWISE",
                self.buttons["rotate_cw"].center,
                self.buttons["rotate_cw"].width,
            )
            self.draw_text(
                "COUNTER-CLOCKWISE",
                self.buttons["rotate_ccw"].center,
                self.buttons["rotate_ccw"].width,
            )

    def click_handler(self, event):
        if self.game_over or self.current_player != 1:
            return

        x, y = event.pos
        if self.move_stage == 0:  # Place piece
            if x > 600:
                return  # clicks on control area
            col = x // 100
            row = y // 100
            if 0 <= row < 6 and 0 <= col < 6 and self.board[row][col] == 0:
                self.temp_piece = (row, col)
                self.board[row][col] = 1
                self.move_stage = 1
                self.draw_board()

        elif self.move_stage == 1:  # Select block
            if x > 600:
                return
            # which block was clicked
            block_x = 0 if x < 300 else 1
            block_y = 0 if y < 300 else 1
            self.selected_block = block_y * 2 + block_x
            self.move_stage = 2
            self.show_rotation_buttons()
            self.highlight_selected_block()

        elif self.move_stage == 2:  # Rotate
            if self.buttons["rotate_cw"].collidepoint(event.pos):
                self.apply_rotation("cw")
            if self.buttons["rotate_ccw"].collidepoint(event.pos):
                self.apply_rotation("ccw")

    def apply_rotation(self, direction):
        self.rotate_block(self.board, self.selected_block, direction)
        self.current_player = -1
        self.move_stage = 0
        self.selected_block = None
        self.temp_piece = None
        self.hide_rotation_buttons()
        self.draw_board()
        pygame.display.flip()
        self.check_game_over()
        pygame.time.delay(1000)
        self.play_computer_move()

    def highlight_selected_block(self):
        colors = [
            (255, 153, 153),
            (153, 255, 153),
            (153, 153, 255),
            (255, 255, 153),
        ]  # RGB colors

        row_start = (self.selected_block // 2) * 3
        col_start = (self.selected_block % 2) * 3

        pygame.draw.rect(
            self.screen,
            colors[self.selected_block],
            (col_start * 100, row_start * 100, 300, 300),
            5,
        )

    def play_computer_move(self):
        move = self.get_computer_move()
        if move and not self.game_over:
            new_board = self.apply_move(self.board, move, -1)
            if new_board is not None:
                self.board = new_board
                self.current_player = 1
                self.draw_board()
                pygame.display.flip()
                self.check_game_over()
            else:
                print("Invalid computer move!")

    def check_game_over(self):
        winner = self.check_winner(self.board)
        if winner is not None:
            self.game_over = True
            self.result = winner
            print("Game over! Result:", winner)
            if self.ui:
                self.show_game_over_message()

    def show_game_over_message(self):
        self.screen.fill((200, 200, 200))
        pygame.draw.rect(self.screen, (255, 255, 255), (100, 200, 500, 200))
        pygame.draw.rect(self.screen, (0, 0, 0), (100, 200, 500, 200), 3)

        result_text = f"Player {self.result} wins!" if self.result != 0 else "Draw!"
        text_surface = self.font_large.render(result_text, True, (255, 0, 0))
        self.screen.blit(text_surface, (250, 250))

        exit_text = self.font_small.render("Click anywhere to exit", True, (0, 0, 0))
        self.screen.blit(exit_text, (230, 350))
        pygame.display.flip()

    def play(self):
        if self.ui:
            running = True
            while running:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        running = False
                    elif event.type == pygame.MOUSEBUTTONDOWN:
                        self.click_handler(event)
                self.draw_board()
                pygame.display.flip()
            pygame.quit()
            return self.result
        else:
            while not self.game_over:
                self.print_board()
                winner = self.check_winner(self.board)
                if winner is not None:
                    return winner

                if self.current_player == 1:
                    move = random.choice(self.get_possible_moves(self.board, 1))
                else:
                    move = self.get_computer_move()

                self.board = self.apply_move(self.board, move, self.current_player)
                self.current_player *= -1
            return self.result

    def print_board(self):
        if self.print == False:
            return
        print("-" * 25)
        for row in self.board:
            print(" ".join(f"{x:2}" for x in row))
        print("-" * 25)

## Analysis

In [4]:
def run_experiment(use_alpha_beta=False, num_games=50, depth=2, show_moves=False):
    results = {
        "wins": 0,
        "ties": 0,
        "losses": 0,
        "times": [],
        "nodes_visited": []
    }
    
    for game_num in range(1, num_games + 1):
        game = PentagoGame(ui=False, print=show_moves, depth=depth, use_alpha_beta_=use_alpha_beta)
        start_time = time.time()
        
        result = game.play()
        
        game_time = time.time() - start_time
        results["times"].append(game_time)
        results["nodes_visited"].append(game.total_visited_nodes)
        
        if result == -1:
            results["wins"] += 1
        elif result == 0:
            results["ties"] += 1
        else:
            results["losses"] += 1
            
        print(f"Game {game_num}: Result={result}, Time={game_time:.2f}s, Nodes={game.total_visited_nodes}")
    
    return results

In [5]:
def print_results(results, title):
    print(f"\n{title} Results:")
    total_games = results["wins"] + results["ties"] + results["losses"]
    print(f"Wins: {results['wins']} ({results['wins']/total_games * 100:.1f}%)")
    print(f"Ties: {results['ties']} ({results['ties']/total_games * 100:.1f}%)")
    print(f"Losses: {results['losses']} ({results['losses']/total_games * 100:.1f}%)")
    print(f"Avg Time: {statistics.mean(results['times']):.4f}s")
    print(f"Avg Nodes Visited: {statistics.mean(results['nodes_visited']):.0f}")

In [ ]:
def test(num_games, depth, show_moves_, test_naive_evaluation=True):    
    print("Running experiments...")
    
    if test_naive_evaluation:
        print("\nRunning regular minimax...")
        regular_results = run_experiment(use_alpha_beta=False, num_games=num_games, depth=depth, show_moves=show_moves_)
        print_results(regular_results, "Regular Minimax")
    
    print("\nRunning alpha-beta pruning...")
    alpha_beta_results = run_experiment(use_alpha_beta=True, num_games=num_games, depth=depth, show_moves=show_moves_)
    print_results(alpha_beta_results, "Alpha-Beta Pruning")
    
    if test_naive_evaluation and regular_results["times"] and alpha_beta_results["times"]:
        print("\nComparison Summary:")
        time_improvement = (1 - statistics.mean(alpha_beta_results["times"])/statistics.mean(regular_results["times"])) * 100
        nodes_improvement = (1 - statistics.mean(alpha_beta_results["nodes_visited"])/statistics.mean(regular_results["nodes_visited"])) * 100
        print(f"Time Improvement: {time_improvement:.1f}% faster")
        print(f"Nodes Visited Improvement: {nodes_improvement:.1f}% reduction")
    else:
        print("\nCould not complete comparison due to failed games")

In [7]:
test(50, 1, show_moves_=False)

Running experiments...

Running regular minimax...
Game 1: Result=-1, Time=0.72s, Nodes=2080
Game 2: Result=-1, Time=0.51s, Nodes=1792
Game 3: Result=-1, Time=0.49s, Nodes=1624
Game 4: Result=-1, Time=0.44s, Nodes=1624
Game 5: Result=-1, Time=0.75s, Nodes=2080
Game 6: Result=-1, Time=0.73s, Nodes=2200
Game 7: Result=-1, Time=0.62s, Nodes=1624
Game 8: Result=-1, Time=1.34s, Nodes=1792
Game 9: Result=-1, Time=1.97s, Nodes=2200
Game 10: Result=-1, Time=2.23s, Nodes=2520
Game 11: Result=-1, Time=1.47s, Nodes=1944
Game 12: Result=-1, Time=1.09s, Nodes=1624
Game 13: Result=-1, Time=1.10s, Nodes=1624
Game 14: Result=-1, Time=1.21s, Nodes=1624
Game 15: Result=-1, Time=1.16s, Nodes=1624
Game 16: Result=-1, Time=1.99s, Nodes=2200
Game 17: Result=-1, Time=1.11s, Nodes=1624
Game 18: Result=-1, Time=1.56s, Nodes=1944
Game 19: Result=-1, Time=1.23s, Nodes=1792
Game 20: Result=-1, Time=1.28s, Nodes=1792
Game 21: Result=-1, Time=1.31s, Nodes=1944
Game 22: Result=-1, Time=1.37s, Nodes=1944
Game 23: Res

In [8]:
test(3, 2, show_moves_=True)

Running experiments...

Running regular minimax...
-------------------------
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
-------------------------
-------------------------
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  1  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
-------------------------
Move took 33.87s, nodes visited: 76440
-------------------------
 0  0  0  0  0  0
 0 -1  0  0  0  0
 0  0  0  0  1  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
-------------------------
-------------------------
 0  0  0  0  0  0
 0 -1  0  0  0  0
 0  0  0  0  1  0
 0  0  0  0  0  0
 0  1  0  0  0  0
 0  0  0  0  0  0
-------------------------
Move took 34.79s, nodes visited: 67848
-------------------------
 0  0  0  0  0  0
 0 -1  0  0 -1  0
 0  0  0  0  1  0
 0  0  0  0  0  0
 0  1  0  0  0  0
 0  0  0  0  0  0
-------------------------
-------------------------
 0  0  0  0  0  0
 0 -1  0  1 -1  0
 0  1  0 

In [9]:
test(1, 3, show_moves_=True, test_naive_evaluation=False)

Running experiments...

Running alpha-beta pruning...
-------------------------
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
-------------------------
-------------------------
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  1
-------------------------
Move took 43.53s, nodes visited: 194132
-------------------------
 0  0  0  0  0  0
 0 -1  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  1
-------------------------
-------------------------
 0  0  0  0  0  0
 0 -1  0  0  0  0
 0  0  0  0  0  0
 0  0  1  0  0  1
 0  0  0  0  0  0
 0  0  0  0  0  0
-------------------------
Move took 74.72s, nodes visited: 284964
-------------------------
 0  0  0  0  0  0
 0 -1  0  0 -1  0
 0  0  0  0  0  0
 0  0  1  0  0  1
 0  0  0  0  0  0
 0  0  0  0  0  0
-------------------------
-------------------------
 0  0  0  0  0  0
 0 -1  0  0 -1  1
 0  

UnboundLocalError: cannot access local variable 'regular_results' where it is not associated with a value

## Questions

### ***1- The Tradeoff Between Depth and Complexity***

An AI's ability to make winning decisions improves as it searches deeper into possible future moves. This allows the system to anticipate strategic advantages several turns ahead.  

However, this improvement comes at a steep cost: the sheer number of calculations required grows at an exponential rate. Each level of added depth dramatically increases the number of positions that must be evaluated, which leads to longer processing times. 

As a result, while deeper searches enhance decision-making, they also demand greater computational resources, making efficiency a crucial concern.  

### ***2- Structuring Searches for Maximum Efficiency***

The way in which an AI evaluates potential moves can be optimized to accelerate decision-making. By prioritizing strong candidates early on, the system can discard weak options faster.  

One method involves applying fast heuristics that assign preliminary scores to moves before fully analyzing them. This lets the algorithm focus on high-potential strategies first.  

Additionally, precomputed databases of known winning patterns can be leveraged to speed up evaluations—if a move leads to a previously identified favorable scenario, the AI can immediately recognize its value without extra calculations.  

### ***3- Evolution of the Branching Factor in Gameplay***

At the start of a game, players typically have a wide array of choices, meaning the branching factor—the number of possible moves per turn—is at its peak. As the match progresses, available moves become more constrained due to board limitations, leading to a steady decline in the branching factor.  

Eventually, near the endgame, options may be reduced to fewer than ten possible moves per turn. This shift has strategic implications: strong players can force the game into a stage where decision-making is simpler, reducing the opponent’s ability to mount complex counterplays.  

### ***4- Alpha-Beta Pruning and Search Optimization***

One of the most effective techniques for enhancing AI performance in decision-making games is alpha-beta pruning. This method refines the search process by cutting off branches that cannot influence the final outcome.  

Suppose a maximizing player has already determined a move that guarantees a value of 10. If, in a lower-level evaluation, a minimizing player encounters a move capped at a value of 2, the system can immediately disregard further exploration of that path, since the optimal player would never select it.  

By eliminating unnecessary computations, pruning significantly speeds up AI reasoning without sacrificing accuracy.  

### ***5- Expectimax and Handling Randomized Opponents***

The traditional Minimax strategy assumes opponents always play optimally.  

However, in real-world gameplay, especially when randomness is involved, this assumption does not always hold. When faced with unpredictable or probabilistic decision-making—such as a player choosing moves arbitrarily—Minimax can struggle to produce effective strategies.  

A better alternative in such cases is Expectimax, an algorithm that integrates probability into its decision-making model.  
Expectimax accounts for uncertainty by weighing move outcomes based on likelihood rather than strict best-case scenarios, though this comes at the cost of additional computation time.